# Cleaning Agent Model

In [2]:
#%%cmd
#pip install mesa

%matplotlib inline

Microsoft Windows [Versi¢n 10.0.19042.985]
(c) Microsoft Corporation. Todos los derechos reservados.

C:\Users\Isaac\OneDrive\Documentos\Agentes>pip install mesa

C:\Users\Isaac\OneDrive\Documentos\Agentes>
C:\Users\Isaac\OneDrive\Documentos\Agentes>%matplotlib inline

C:\Users\Isaac\OneDrive\Documentos\Agentes>

"%matplotlib" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [3]:
# Mesa
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

# Matplotlib
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Libraries
import numpy as np
import pandas as pd
import time
import datetime
import random
from contextlib import contextmanager
import threading
import _thread

# Functions

In [4]:
def get_grid(model):
    '''
    Stores the grid content in an array.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        grid[x][y] = cell_content[0].state
    return grid

@contextmanager
def time_limit(seconds):
    timer = threading.Timer(seconds, lambda: _thread.interrupt_main())
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        raise Exception("Timed out for operation")
        timer.cancel()

# Create the agent
Create an agent that will move randomly in his Moore neighborhood, once he has moved he will clean (change the Tile state from 1 "dirty" to 0 "clean) the tile he is on. 

In [5]:
class CleaningAgent(Agent):
    """ An agent that cleans. """
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.state = 2
        self.moves = 0

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def clean(self):
        dirty_tiles = self.model.grid.get_cell_list_contents([self.pos])
        if len(dirty_tiles) > 1:
            for tile in dirty_tiles:
                if (tile.state == 1):
                    tile.state = 0

    def step(self):
        self.move()
        self.clean()
        self.moves += 1

# Create tiles
Create the floor tiles that will be in each coordinate of the grid, they will be randomly distributed and their state will depend on the density of dirty tiles.

<div class="alert alert-block alert-warning">
<b>Explanation:</b> The floor tiles are not agents in the model, however Mesa can only model agents, so for visualization purposes the floor tiles are defined in the code as agents.
</div>

In [6]:
class Tile(Agent):
    '''
    Represents a floor tile that is either dirty (1) or clean (0).
    '''
    def __init__(self, unique_id, state, model):
        super().__init__(unique_id, model)
        self.state = state

    def step(self):
        pass

# Create the model
The model takes:
- N number of agents that will start the cleaning. (Position is fixed to 1,1 for every agent)
- Height of the grid
- Width of the grid
- Density of dirty tiles.

The environment is a MultiGrid, because that allows Cleaning Agents to be on the same grid cell as the Tile.
The model will run until there are no more tiles left to clean.

In [7]:
class CleaningModel(Model):
    """A model with some number of agents."""
    
    def __init__(self, N, width, height, density):
        # Initialize model parameters
        self.num_agents = N
        self.height = height
        self.width = width
        self.density = density
        
        # Set up model objects
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        # Create cleaning agents
        for j in range(self.num_agents):
            a = CleaningAgent('Cl-'+str(j), self)
            self.schedule.add(a)
            # Add the agent to a 1,1
            x = 1
            y = 1
            self.grid.place_agent(a, (x, y))
                      
        # Create clean tiles and dirty tiles based on density
        for (content, x, y) in self.grid.coord_iter():
            a = Tile((x, y), 0, self)
            if random.random() < self.density:
                a = Tile((x, y), 1, self)
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)

        # Define collector for the entire grid 
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
        # Run the model until it is halted
        self.running = True

    def step(self):
        self.datacollector.collect(self)
        # Halt if no more dirty tiles
        if self.count_type(self, 1) == 0:
            self.running = False
        self.schedule.step()
        
        
    @staticmethod
    def count_type(model, state):
        '''
        Helper method to count agents in a given condition in a given model.
        '''
        count = 0
        for tl in model.schedule.agents:
            if tl.state == state:
                count += 1
        return count

# Model execution

In [8]:
# Define params 
agents = 5
height = 10
width = 10
density = 0.8
tmax = 0.0031

model = CleaningModel(agents, height, width, density)

# Start to measure the time
start_time = time.monotonic()
# Start cleaning with time limit
with time_limit(tmax):
    try:
        model.run_model()
    except:
        print(f'Time out after {tmax}s\n')
    else:
        end_time = time.monotonic()
        t = datetime.timedelta(seconds=end_time - start_time)
        print(f'Execution time: {t.seconds}.0{t.microseconds:}s')

# Print the moves of all agents
for agent in model.schedule.agents:
    if ('Cl' in str(agent.unique_id)):
        print(f'Agent {agent.unique_id} moved {agent.moves} times')
        
# Print % of cleaningness
all_grid = model.datacollector.get_model_vars_dataframe()
cleanp = abs((1 - (all_grid.iloc[-1][0].sum() / 100) )) * 100
print(f'Percentage of cleaned tiles: {cleanp}%')

Time out after 0.0031s

Agent Cl-0 moved 14 times
Agent Cl-1 moved 14 times
Agent Cl-2 moved 14 times
Agent Cl-3 moved 14 times
Agent Cl-4 moved 14 times
Percentage of cleaned tiles: 46.0%


# Visualización

In [9]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [10]:
anim